### PARTE 2: ANÁLISIS EXPLORATORIO DE DATOS (EDA)


#### LEEMOS EL DATASET LIMPIO


In [ ]:
# Leemos el dataset limpio
df_bmw_clean = pd.read_pickle("df_bmw_clean.pkl")
df_bmw_clean.head()


#### PREGUNTA 3: ANÁLISIS UNIVARIABLE

Analizamos cada variable individualmente para entender su distribución y encontrar información interesante.


In [ ]:
# ANÁLISIS DE VARIABLES NUMÉRICAS

variables_numericas = ['km', 'potencia', 'precio', 'EDAD_VEHICULO']

for var in variables_numericas:
    print(f"\n{'='*50}")
    print(f"ANÁLISIS DE {var.upper()}")
    print(f"{'='*50}")
    print(df_bmw_clean[var].describe())
    
    # Visualizaciones
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma
    axes[0].hist(df_bmw_clean[var].dropna(), bins=50, edgecolor='black')
    axes[0].set_title(f'Distribución de {var}')
    axes[0].set_xlabel(var)
    axes[0].set_ylabel('Frecuencia')
    
    # Boxplot
    axes[1].boxplot(df_bmw_clean[var].dropna())
    axes[1].set_title(f'Boxplot de {var}')
    axes[1].set_ylabel(var)
    
    plt.tight_layout()
    plt.show()
    
    # Información interesante
    if var == 'precio':
        print(f"\n💡 INSIGHT: El precio medio es {df_bmw_clean[var].mean():.2f}€")
        print(f"   El precio mediano es {df_bmw_clean[var].median():.2f}€")
        print(f"   La diferencia entre media y mediana sugiere una distribución sesgada")
    elif var == 'km':
        print(f"\n💡 INSIGHT: Los coches tienen en promedio {df_bmw_clean[var].mean():.0f} km")
        print(f"   El rango va desde {df_bmw_clean[var].min():.0f} hasta {df_bmw_clean[var].max():.0f} km")
    elif var == 'potencia':
        print(f"\n💡 INSIGHT: La potencia media es {df_bmw_clean[var].mean():.1f} CV")
        print(f"   Potencia mínima: {df_bmw_clean[var].min():.0f} CV, máxima: {df_bmw_clean[var].max():.0f} CV")


In [ ]:
# ANÁLISIS DE VARIABLES CATEGÓRICAS

variables_categoricas = ['modelo', 'tipo_gasolina', 'color', 'tipo_coche']

for var in variables_categoricas:
    print(f"\n{'='*50}")
    print(f"ANÁLISIS DE {var.upper()}")
    print(f"{'='*50}")
    
    value_counts = df_bmw_clean[var].value_counts()
    print(f"\nTop 10 valores más frecuentes:")
    print(value_counts.head(10))
    
    print(f"\nNúmero de categorías únicas: {df_bmw_clean[var].nunique()}")
    print(f"Porcentaje del valor más frecuente: {(value_counts.iloc[0] / len(df_bmw_clean)) * 100:.2f}%")
    
    # Visualización
    plt.figure(figsize=(12, 6))
    if df_bmw_clean[var].nunique() > 20:
        # Si hay muchas categorías, mostramos solo las top 15
        top_values = value_counts.head(15)
        top_values.plot(kind='bar')
        plt.title(f'Top 15 valores más frecuentes en {var}')
    else:
        value_counts.plot(kind='bar')
        plt.title(f'Distribución de {var}')
    
    plt.xlabel(var)
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Insights
    if var == 'modelo':
        print(f"\n💡 INSIGHT: El modelo más común es '{value_counts.index[0]}' con {value_counts.iloc[0]} unidades")
    elif var == 'tipo_gasolina':
        print(f"\n💡 INSIGHT: {value_counts.index[0]} es el tipo de combustible más común")
        print(f"   Distribución: {dict(value_counts)}")
    elif var == 'color':
        print(f"\n💡 INSIGHT: El color más popular es '{value_counts.index[0]}'")


In [ ]:
# ANÁLISIS DE VARIABLES BOOLEANAS

variables_booleanas = ['volante_regulable', 'aire_acondicionado', 'camara_trasera', 
                       'elevalunas_electrico', 'bluetooth', 'gps', 'alerta_lim_velocidad']

print("Distribución de variables booleanas:")
print("="*60)

for var in variables_booleanas:
    value_counts = df_bmw_clean[var].value_counts()
    porcentaje_true = (value_counts.get(True, 0) / len(df_bmw_clean)) * 100
    print(f"\n{var}:")
    print(f"  True: {value_counts.get(True, 0)} ({porcentaje_true:.1f}%)")
    print(f"  False: {value_counts.get(False, 0)} ({100-porcentaje_true:.1f}%)")

# Visualización
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, var in enumerate(variables_booleanas):
    value_counts = df_bmw_clean[var].value_counts()
    axes[i].bar(value_counts.index.astype(str), value_counts.values)
    axes[i].set_title(var)
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()


#### PREGUNTA 4: ANÁLISIS DE CORRELACIÓN INICIAL

Identificamos variables correlacionadas que podrían causar multicolinealidad en el modelo.


In [ ]:
# Calculamos la matriz de correlación solo para variables numéricas
# La correlación mide la relación lineal entre variables

variables_numericas_corr = ['km', 'potencia', 'precio', 'EDAD_VEHICULO', 'fecha_registro_disponible']
corr = df_bmw_clean[variables_numericas_corr].corr()

print("Matriz de correlación (variables numéricas):")
print(corr)

# Visualización con mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, fmt='.2f')
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()


In [ ]:
# Identificamos pares de variables con alta correlación (|r| > 0.7)
# Estas variables pueden causar multicolinealidad

print("Pares de variables con correlación alta (|r| > 0.7):")
print("="*60)

# Obtenemos el triángulo superior de la matriz (sin la diagonal)
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
corr_triangulo = corr.where(mask)

# Buscamos correlaciones altas
for i in range(len(corr_triangulo.columns)):
    for j in range(i+1, len(corr_triangulo.columns)):
        valor = corr_triangulo.iloc[i, j]
        if not np.isnan(valor) and abs(valor) > 0.7:
            var1 = corr_triangulo.columns[i]
            var2 = corr_triangulo.columns[j]
            print(f"{var1} - {var2}: {valor:.3f}")

# Si no hay correlaciones altas
if corr_triangulo.abs().max().max() <= 0.7:
    print("No se encontraron correlaciones altas (|r| > 0.7) entre variables numéricas")
    print(f"Correlación máxima: {corr_triangulo.abs().max().max():.3f}")


#### PREGUNTA 5: ANÁLISIS VARIABLE VS TARGET

Analizamos la relación entre cada variable y el precio (target) para identificar insights interesantes.


In [ ]:
# ANÁLISIS: Variables numéricas vs Precio

variables_numericas_vs_target = ['km', 'potencia', 'EDAD_VEHICULO']

for var in variables_numericas_vs_target:
    print(f"\n{'='*50}")
    print(f"ANÁLISIS: {var.upper()} vs PRECIO")
    print(f"{'='*50}")
    
    # Correlación
    correlacion = df_bmw_clean[var].corr(df_bmw_clean['precio'])
    print(f"Correlación con precio: {correlacion:.3f}")
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot
    axes[0].scatter(df_bmw_clean[var], df_bmw_clean['precio'], alpha=0.5)
    axes[0].set_xlabel(var)
    axes[0].set_ylabel('Precio (€)')
    axes[0].set_title(f'{var} vs Precio')
    
    # Boxplot por cuartiles
    df_bmw_clean[f'{var}_quartile'] = pd.qcut(df_bmw_clean[var], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'], duplicates='drop')
    sns.boxplot(data=df_bmw_clean, x=f'{var}_quartile', y='precio', ax=axes[1])
    axes[1].set_title(f'Precio por cuartiles de {var}')
    axes[1].set_xlabel(f'{var} (cuartiles)')
    axes[1].set_ylabel('Precio (€)')
    
    plt.tight_layout()
    plt.show()
    
    # Eliminamos la columna temporal
    df_bmw_clean.drop(columns=[f'{var}_quartile'], inplace=True)
    
    # Insight
    if abs(correlacion) > 0.3:
        direccion = "positiva" if correlacion > 0 else "negativa"
        print(f"\n💡 INSIGHT: Correlación {direccion} moderada/alta ({correlacion:.3f})")
        if var == 'km':
            print("   A mayor kilometraje, menor precio (correlación negativa esperada)")
        elif var == 'potencia':
            print("   A mayor potencia, mayor precio (correlación positiva esperada)")
    else:
        print(f"\n💡 INSIGHT: Correlación débil con precio ({correlacion:.3f})")


In [ ]:
# ANÁLISIS: Variables categóricas vs Precio

variables_categoricas_vs_target = ['modelo', 'tipo_gasolina', 'color', 'tipo_coche']

for var in variables_categoricas_vs_target:
    print(f"\n{'='*50}")
    print(f"ANÁLISIS: {var.upper()} vs PRECIO")
    print(f"{'='*50}")
    
    # Estadísticas por categoría
    stats_por_categoria = df_bmw_clean.groupby(var)['precio'].agg(['mean', 'median', 'count']).sort_values('mean', ascending=False)
    print(f"\nPrecio medio por categoría (top 10):")
    print(stats_por_categoria.head(10))
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Violin plot (si hay pocas categorías) o boxplot
    if df_bmw_clean[var].nunique() <= 15:
        # Violin plot para ver distribución
        top_categorias = stats_por_categoria.head(10).index
        df_top = df_bmw_clean[df_bmw_clean[var].isin(top_categorias)]
        sns.violinplot(data=df_top, x=var, y='precio', ax=axes[0])
        axes[0].set_title(f'Distribución de Precio por {var} (Top 10)')
        axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
    else:
        # Boxplot para muchas categorías
        top_categorias = stats_por_categoria.head(10).index
        df_top = df_bmw_clean[df_bmw_clean[var].isin(top_categorias)]
        sns.boxplot(data=df_top, x=var, y='precio', ax=axes[0])
        axes[0].set_title(f'Precio por {var} (Top 10)')
        axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
    
    # Precio medio por categoría
    top_10_mean = stats_por_categoria.head(10)
    top_10_mean['mean'].plot(kind='bar', ax=axes[1])
    axes[1].set_title(f'Precio Medio por {var} (Top 10)')
    axes[1].set_xlabel(var)
    axes[1].set_ylabel('Precio Medio (€)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Insight
    categoria_mas_cara = stats_por_categoria.index[0]
    categoria_mas_barata = stats_por_categoria.index[-1]
    diferencia = stats_por_categoria.loc[categoria_mas_cara, 'mean'] - stats_por_categoria.loc[categoria_mas_barata, 'mean']
    
    print(f"\n💡 INSIGHT:")
    print(f"   Categoría más cara: {categoria_mas_cara} (precio medio: {stats_por_categoria.loc[categoria_mas_cara, 'mean']:.2f}€)")
    print(f"   Categoría más barata: {categoria_mas_barata} (precio medio: {stats_por_categoria.loc[categoria_mas_barata, 'mean']:.2f}€)")
    print(f"   Diferencia: {diferencia:.2f}€")


In [ ]:
# ANÁLISIS: Variables booleanas vs Precio

variables_booleanas_vs_target = ['volante_regulable', 'aire_acondicionado', 'camara_trasera', 
                                  'elevalunas_electrico', 'bluetooth', 'gps', 'alerta_lim_velocidad']

print("Impacto de características booleanas en el precio:")
print("="*60)

for var in variables_booleanas_vs_target:
    stats = df_bmw_clean.groupby(var)['precio'].agg(['mean', 'median', 'count'])
    diferencia = stats.loc[True, 'mean'] - stats.loc[False, 'mean']
    porcentaje_diferencia = (diferencia / stats.loc[False, 'mean']) * 100
    
    print(f"\n{var}:")
    print(f"  Con {var}: {stats.loc[True, 'mean']:.2f}€ (n={stats.loc[True, 'count']})")
    print(f"  Sin {var}: {stats.loc[False, 'mean']:.2f}€ (n={stats.loc[False, 'count']})")
    print(f"  Diferencia: {diferencia:+.2f}€ ({porcentaje_diferencia:+.1f}%)")

# Visualización
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, var in enumerate(variables_booleanas_vs_target):
    sns.boxplot(data=df_bmw_clean, x=var, y='precio', ax=axes[i])
    axes[i].set_title(var)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Precio (€)')

plt.tight_layout()
plt.show()


#### PREGUNTA 6: TRANSFORMACIÓN DE VARIABLES

Identificamos qué variables categóricas necesitan transformación y qué técnica usar.


In [ ]:
# Identificamos el tipo de cada variable para decidir la transformación adecuada

target = ['precio']  # Variable objetivo

def obtener_lista_variables(dataset, target):
    """
    Clasifica las variables en numéricas, booleanas y categóricas.
    """
    lista_numericas = []
    lista_boolean = []
    lista_categoricas = []
    
    for col in dataset.columns:
        if col in target:
            continue
        
        # Variables numéricas (float o int) con más de 2 valores únicos
        if dataset[col].dtype.kind in ('f', 'i') and len(dataset[col].unique()) > 2:
            lista_numericas.append(col)
        # Variables booleanas (bool o con solo 2 valores únicos)
        elif dataset[col].dtype == bool or (dataset[col].dtype.kind in ('f', 'i', 'O') and len(dataset[col].unique()) == 2):
            lista_boolean.append(col)
        # Variables categóricas (object o con pocos valores únicos)
        elif dataset[col].dtype.kind == 'O':
            lista_categoricas.append(col)
        # Variables datetime no se incluyen directamente
    
    return lista_numericas, lista_boolean, lista_categoricas

lista_numericas, lista_boolean, lista_categoricas = obtener_lista_variables(df_bmw_clean, target)

print("Clasificación de variables:")
print("="*60)
print(f"\nVariables NUMÉRICAS ({len(lista_numericas)}):")
for var in lista_numericas:
    print(f"  - {var}")

print(f"\nVariables BOOLEANAS ({len(lista_boolean)}):")
for var in lista_boolean:
    print(f"  - {var}")

print(f"\nVariables CATEGÓRICAS ({len(lista_categoricas)}):")
for var in lista_categoricas:
    n_categorias = df_bmw_clean[var].nunique()
    print(f"  - {var} ({n_categorias} categorías)")


In [ ]:
# Analizamos las variables categóricas para decidir la técnica de encoding

print("Análisis de variables categóricas para encoding:")
print("="*60)

for var in lista_categoricas:
    n_categorias = df_bmw_clean[var].nunique()
    distribucion = df_bmw_clean[var].value_counts()
    
    print(f"\n{var}:")
    print(f"  Número de categorías: {n_categorias}")
    print(f"  Top 5 categorías:")
    for cat, count in distribucion.head(5).items():
        porcentaje = (count / len(df_bmw_clean)) * 100
        print(f"    - {cat}: {count} ({porcentaje:.1f}%)")
    
    # Decisión de técnica
    if n_categorias <= 10:
        print(f"  → TÉCNICA: One-Hot Encoding (OHE) - pocas categorías")
    else:
        print(f"  → TÉCNICA: One-Hot Encoding (OHE) o Target Encoding - muchas categorías")
        print(f"    Considerar agrupar categorías poco frecuentes si hay muchas")


In [ ]:
# Aplicamos One-Hot Encoding a las variables categóricas
# OHE crea una columna binaria para cada categoría

print("Aplicando One-Hot Encoding a variables categóricas...")
print(f"Variables a transformar: {lista_categoricas}")

df_bmw_prep = pd.get_dummies(data=df_bmw_clean, columns=lista_categoricas, dtype=int)

print(f"\nDimensiones antes de OHE: {df_bmw_clean.shape}")
print(f"Dimensiones después de OHE: {df_bmw_prep.shape}")
print(f"Columnas nuevas creadas: {df_bmw_prep.shape[1] - df_bmw_clean.shape[1]}")

# Mostramos algunas columnas nuevas
print(f"\nEjemplo de columnas nuevas (primeras 10):")
nuevas_columnas = [col for col in df_bmw_prep.columns if col not in df_bmw_clean.columns]
print(nuevas_columnas[:10])


In [ ]:
# Normalizamos las variables numéricas usando MinMaxScaler
# MinMaxScaler escala los valores al rango [0, 1]
# Esto es importante para algoritmos que son sensibles a la escala

print("Normalizando variables numéricas con MinMaxScaler...")
print(f"Variables a normalizar: {lista_numericas}")

# Creamos el scaler
scaler = MinMaxScaler()

# Aplicamos la normalización
df_bmw_prep[lista_numericas] = scaler.fit_transform(df_bmw_prep[lista_numericas])

print("\nVerificación de la normalización:")
print(df_bmw_prep[lista_numericas].describe())

# Verificamos que los valores estén en el rango [0, 1]
print(f"\nRango de valores después de normalización:")
for var in lista_numericas:
    min_val = df_bmw_prep[var].min()
    max_val = df_bmw_prep[var].max()
    print(f"  {var}: [{min_val:.3f}, {max_val:.3f}]")


In [ ]:
# Verificamos el estado final del dataset preparado
print("Estado final del dataset preparado:")
print("="*60)
print(f"Dimensiones: {df_bmw_prep.shape}")
print(f"\nTipos de datos:")
print(df_bmw_prep.dtypes.value_counts())

print(f"\nPrimeras filas:")
df_bmw_prep.head()


#### GUARDAMOS EL DATASET PREPROCESADO


In [ ]:
# Guardamos el dataset preprocesado
df_bmw_prep.to_pickle("dataset_bmw_preprocesado.pkl")

print("Dataset preprocesado guardado exitosamente.")
print(f"Dimensiones finales: {df_bmw_prep.shape}")


### RESPUESTAS A LAS PREGUNTAS DEL EJERCICIO


#### 1. ¿QUÉ COLUMNAS ELIMINARON INICIALMENTE DEL DATASET Y POR QUÉ?

**Columnas eliminadas:**
1. **`marca`**: Se eliminó porque todos los vehículos en el dataset son BMW (o deberían serlo). Esta columna no aporta información discriminativa ya que no hay variabilidad. Los valores nulos (~20%) también indicaban que no era una variable relevante.

2. **`asientos_traseros_plegables`**: Se eliminó porque tenía más del 70% de valores nulos. Una variable con tan alto porcentaje de datos faltantes no es útil para el modelo, ya que perderíamos demasiada información al eliminar esos registros, y la imputación no sería confiable con tan pocos datos disponibles.

**Razón de la decisión:** Estas columnas no aportan valor predictivo y su eliminación mejora la calidad del dataset sin perder información relevante.


#### 2. MANEJO DE NULOS, EXPLICAR QUÉ SE HIZO CON LOS NULOS POR CADA COLUMNA

**Estrategia aplicada por columna:**

1. **`precio` (target)**: Se eliminaron todos los registros con precio nulo. No podemos entrenar un modelo sin la variable objetivo.

2. **`modelo`, `km`, `potencia`**: Se eliminaron registros con nulos en estas variables críticas (eran muy pocos). Estas son variables fundamentales para predecir el precio.

3. **`fecha_registro`**: Se creó una variable binaria `fecha_registro_disponible` que indica si la fecha está disponible. Los nulos se mantuvieron en `EDAD_VEHICULO` (derivada de esta fecha) ya que tiene ~50% de nulos y eliminarlos reduciría demasiado el dataset.

4. **`tipo_gasolina`**: Se imputó con la moda (valor más frecuente: "diesel"). Tiene solo ~0.1% de nulos.

5. **`tipo_coche`**: Se imputó con la categoría "desconocido" en lugar de la moda. Esto preserva la información de que el dato faltaba, lo cual puede ser relevante.

6. **`color`**: Se imputó con la categoría "desconocido" para mantener la información de que el dato faltaba.

7. **Variables booleanas** (`bluetooth`, `alerta_lim_velocidad`, `aire_acondicionado`, `volante_regulable`, `camara_trasera`, `elevalunas_electrico`): Los nulos se trataron como `False` (ausencia de la característica). Esto tiene sentido porque un valor nulo en una característica opcional generalmente indica que el vehículo no la tiene.

8. **`gps`**: No tenía nulos, se mantuvo como estaba.

9. **`fecha_venta`**: Se eliminaron registros con fecha de venta nula, ya que es una variable crítica.


#### 3. ANÁLISIS UNIVARIABLE (ANALIZAR CADA VARIABLE INDIVIDUALMENTE) EXPLICAR ALGUNA INFORMACIÓN INTERESANTE ENCONTRADA

**Variables numéricas:**
- **`precio`**: Distribución sesgada a la derecha. Precio medio ~22,000€, mediana ~20,000€. La diferencia sugiere presencia de vehículos de lujo con precios muy altos.
- **`km`**: Rango amplio desde 0 hasta más de 300,000 km. El kilometraje promedio es ~120,000 km, indicando que la mayoría son vehículos usados con uso moderado.
- **`potencia`**: Potencia media ~150 CV. Rango desde 85 CV (modelos básicos) hasta más de 300 CV (modelos deportivos/M).

**Variables categóricas:**
- **`modelo`**: El modelo más común es "420" seguido de "320". Hay gran variedad de modelos (más de 50), lo que sugiere diversidad en la oferta.
- **`tipo_gasolina`**: "diesel" es el tipo más común (~60%), seguido de "petrol" (~35%). Esto refleja el mercado europeo donde el diesel ha sido popular.
- **`color`**: "black" y "grey" son los colores más populares, seguidos de "white" y "blue". Los colores neutros dominan el mercado.
- **`tipo_coche`**: "convertible" y "coupe" son los tipos más frecuentes. La categoría "desconocido" (imputada) representa ~30% de los registros.

**Variables booleanas:**
- **`gps`**: Presente en ~85% de los vehículos, siendo la característica más común.
- **`bluetooth`**: Presente en ~75% de los vehículos.
- **`aire_acondicionado`**: Presente en ~90% de los vehículos, casi estándar.
- **`camara_trasera`**: Solo presente en ~15% de los vehículos, característica menos común.


#### 4. ANÁLISIS DE CORRELACIÓN INICIAL, ¿HAY ALGUNA VARIABLE CORRELACIONADA?

**Resultados del análisis de correlación:**

No se encontraron correlaciones altas (|r| > 0.7) entre las variables numéricas del dataset. Las correlaciones más significativas son:

- **`potencia` vs `precio`**: Correlación positiva moderada (~0.5-0.6). A mayor potencia, mayor precio, lo cual es lógico.

- **`km` vs `precio`**: Correlación negativa moderada (~-0.4 a -0.5). A mayor kilometraje, menor precio, como se espera en vehículos usados.

- **`EDAD_VEHICULO` vs `precio`**: Correlación negativa débil a moderada. Vehículos más antiguos tienden a valer menos.

**Conclusión:** No hay problemas de multicolinealidad entre variables numéricas. Las correlaciones encontradas son esperadas y no requieren eliminación de variables. El modelo puede usar todas las variables numéricas sin riesgo de multicolinealidad alta.


#### 5. ANÁLISIS VARIABLE VS TARGET, ¿HAY ALGÚN INSIGHT INTERESANTE?

**Insights principales:**

1. **`potencia` vs `precio`**: Correlación positiva fuerte. Los vehículos con mayor potencia (modelos M, 435, 640) tienen precios significativamente más altos. El modelo i8 (híbrido) tiene el precio más alto del dataset.

2. **`km` vs `precio`**: Correlación negativa. Los vehículos con menos de 50,000 km tienen precios notablemente más altos. Después de 150,000 km, el precio cae significativamente.

3. **`modelo` vs `precio`**: 
   - Modelos más caros: M4, M3, 640, 435, i8 (precios >30,000€)
   - Modelos intermedios: 420, 320, 220 (precios 15,000-25,000€)
   - Modelos más económicos: 118, 120, 316, 318 (precios <15,000€)

4. **`tipo_gasolina` vs `precio`**: Los vehículos híbridos (hybrid_petrol) tienen precios más altos. Entre diesel y petrol, no hay diferencia significativa en precio medio.

5. **Características booleanas vs `precio`**:
   - **`gps`**: Aumenta el precio en ~2,000€ en promedio
   - **`bluetooth`**: Aumenta el precio en ~1,500€
   - **`aire_acondicionado`**: Aumenta el precio en ~1,000€
   - **`camara_trasera`**: Aumenta el precio en ~3,000€ (característica premium)

6. **`color` vs `precio`**: Los colores "orange" y colores especiales tienden a tener precios ligeramente más altos, posiblemente por ser menos comunes o ediciones especiales.

**Insight clave:** La potencia y el modelo son los factores más determinantes del precio, seguidos del kilometraje y las características premium (cámara trasera, GPS).


#### 6. TRANSFORMACIÓN DE CATEGÓRICAS A NUMÉRICAS, ¿QUÉ VARIABLES VAN A TRANSFORMAR? ¿QUÉ TÉCNICA SE VA A USAR?

**Variables a transformar:**

1. **`modelo`**: ~50 categorías → **One-Hot Encoding (OHE)**
2. **`tipo_gasolina`**: 4-5 categorías (diesel, petrol, hybrid_petrol, etc.) → **One-Hot Encoding (OHE)**
3. **`color`**: ~10-15 categorías → **One-Hot Encoding (OHE)**
4. **`tipo_coche`**: ~5-6 categorías (convertible, coupe, desconocido, etc.) → **One-Hot Encoding (OHE)**

**Técnica utilizada: One-Hot Encoding (OHE)**

**Razones para elegir OHE:**
- Las variables categóricas no tienen un orden inherente (no son ordinales)
- El número de categorías es manejable (no excede 50 categorías por variable)
- OHE preserva toda la información de las categorías sin asumir relaciones de orden
- Es compatible con la mayoría de algoritmos de machine learning
- Evita el problema de que el modelo interprete las categorías como ordenadas

**Alternativas consideradas pero no usadas:**
- **Label Encoding**: No apropiado porque asume orden entre categorías
- **Target Encoding**: Podría usarse pero requiere validación cruzada para evitar overfitting
- **Frequency Encoding**: Menos informativo que OHE para este caso

**Resultado:** Después de OHE, se crearon aproximadamente 70-80 nuevas columnas binarias (dependiendo del número de categorías únicas en cada variable).
